In [1]:
import sys
import os
import numpy as np
import pandas as pd
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical

In [2]:
df=pd.read_csv('fer2013.csv')

In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10254 entries, 0 to 10253
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  10254 non-null  int64 
 1   pixels   10254 non-null  object
 2   Usage    10253 non-null  object
dtypes: int64(1), object(2)
memory usage: 240.5+ KB
None


In [4]:
print(df["Usage"])

0        Training
1        Training
2        Training
3        Training
4        Training
           ...   
10249    Training
10250    Training
10251    Training
10252    Training
10253         NaN
Name: Usage, Length: 10254, dtype: object


In [5]:
print(df.head())

   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training


In [6]:
X_train,train_y,X_test,test_y=[],[],[],[]

In [7]:
for index,row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
          X_train.append(np.array(val,'float32'))
          train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
          X_test.append(np.array(val,'float32'))
    except:
        print(f"error occured at index:{index} and row:{row}")
  
print(f"X_train sample data:{X_train[0:2]}")
print(f"train_y sample data: {train_y[0:2]}")
print(f"X_test sample data: {X_test[0:2]}")
print(f"test_y sample data: {test_y[0:2]}")

error occured at index:10253 and row:emotion                                                    0
pixels     87 90 92 92 91 86 69 28 12 16 16 53 104 121 12...
Usage                                                    NaN
Name: 10253, dtype: object
X_train sample data:[array([ 70.,  80.,  82., ..., 106., 109.,  82.], dtype=float32), array([151., 150., 147., ..., 193., 183., 184.], dtype=float32)]
train_y sample data: [0, 0]
X_test sample data: []
test_y sample data: []


In [8]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 200
width, height = 48, 48

In [9]:
X_train=np.array(X_train,'float32')
train_y=np.array(train_y,'float32')
X_test=np.array(X_test,'float32')
test_y=np.array(test_y,'float32')

In [10]:
train_y= to_categorical(train_y, num_classes=num_labels)
test_y= to_categorical(test_y, num_classes=num_labels)

In [11]:
#normalizing data

X_train = X_train-np.mean(X_train,axis=0)
X_train = X_train/np.std(X_train,axis=0)

X_test = X_test-np.mean(X_test,axis=0)
X_test = X_test/np.std(X_test,axis=0)


C:\Users\LENOVO\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\LENOVO\anaconda3\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\LENOVO\anaconda3\lib\site-packages\numpy\core\_methods.py:233: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\LENOVO\anaconda3\lib\site-packages\numpy\core\_methods.py:194: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
C:\Users\LENOVO\anaconda3\lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


In [12]:
X_train = X_train.reshape(X_train.shape[0],width,height,1)

X_test = X_test.reshape(X_test.shape[0],width,height,1)

In [13]:
# print(f"shape:{X_train.shape}")
##designing the cnn
#1st convolution layer

model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

In [14]:
#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

In [15]:
#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

model.add(Flatten())



In [16]:
#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax'))

# model.summary()

In [17]:
from tensorflow.keras.losses import categorical_crossentropy

In [18]:
#Compliling the model
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [ ]:
#Training the model
model.fit(X_train, train_y,
          batch_size=batch_size,
          epochs=100,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True)

In [ ]:
import os
import cv2
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image

In [ ]:
#model = model_from_json(open("fer.json", "r").read())
#load weights
#model.load_weights('fer.h5')
model.save('Final_model_95p07.h5')

In [ ]:
import cv2
face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
import os,random
import cv2
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image
import subprocess

from collections import Counter

cap=cv2.VideoCapture(-1)
if not cap.isOpened():
  cap = cv2.VideoCapture(0)
if not cap.isOpened():
  raise IOError("Cannot open webcam")
while True:
    ret,test_img=cap.read()# captures frame and returns boolean value and captured image
    if not ret:
        continue
    gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)


    for (x,y,w,h) in faces_detected:
        cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=7)
        roi_gray=gray_img[y:y+w,x:x+h]#cropping region of interest i.e. face area from  image
        roi_gray=cv2.resize(roi_gray,(48,48))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels /= 255

        predictions = model.predict(img_pixels)

        #find max indexed array
        max_index = np.argmax(predictions[0])

        emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
        predicted_emotion = emotions[max_index]

        cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
       

    resized_img = cv2.resize(test_img, (1000, 700))
    cv2.imshow('Facial emotion analysis ',resized_img)
    #playsound('C:\\real time face detection and music playing accordingly\\songs\\happy songs\\Gallan Goodiyaan.mp3')
    #os.startfile('C:\\real time face detection and music playing accordingly\\songs\\happy songs\\Gallan Goodiyaan.mp3')


    if cv2.waitKey(10) == ord('q'):#wait until 'q' key is pressed
        break

cap.release()
cv2.destroyAllWindows